In [1]:
# imports block
import pandas as pd
import folium

In [12]:
# datetime filter block
date = input("give date in format yyyy-mm-dd\n")
time_start = input("give time window start in format hh:mm\n")
time_end = input("give time window end in format hh:mm\n")
time_start = date + " " + time_start
time_end = date + " " + time_end

give date in format yyyy-mm-dd
2019-01-11
give time window start in format hh:mm
12:00
give time window end in format hh:mm
22:00


In [15]:
#load pickup data here
pickup_times = pd.read_csv('data/pickup_times.csv', header=0)

# convert timestamp to dataframe timeformat
pickup_times['iso_8601_timestamp'] = pd.to_datetime(pickup_times['iso_8601_timestamp'])

# filter columns
pickup_filter = pickup_times[(pickup_times['iso_8601_timestamp'] > time_start) & (pickup_times['iso_8601_timestamp'] < time_end)]

# calculate median
pickup_median = pickup_filter.groupby(['location_id']).median()

# load locations data
locations = pd.read_csv('data/locations.csv', header=0)

#merge median and location data
times_locations = pickup_median.merge(locations, left_index=True, right_index=True)

times_locations.to_csv("output.csv", encoding='utf-8', index=False, columns=['location_id','pickup_time'])

#Initialize map, add the points
folium_map = folium.Map(location=[60.1699, 24.9384], zoom_start=13, tiles="CartoDB dark_matter")
for row in times_locations.itertuples():
    folium.Circle([row.latitude, row.longitude], 100, fill=True).add_child(folium.Popup('median pickup time here is:' + str(row.pickup_time))).add_to(folium_map)
    
# draw map
folium_map